In [1]:
using NBInclude
@nbinclude("Hofstadter Single Particle in Julia.ipynb")

Hofstadter_SP (generic function with 1 method)

In [2]:
using QuantumOptics

In [3]:
function Hofstadter_SP_Op(Nx, Ny, alpha, Basis)
    
    H_SP = Hofstadter_SP(Nx,Ny,alpha, 0)
    
    N = Nx*Ny
    
    H = SparseOperator(Basis)

    for m in 1:N
        for n in 1:N
            H = H + H_SP[m,n] * transition(Basis, m, n)
        end
    end
    
    return H
end

Hofstadter_SP_Op (generic function with 1 method)

1) Hofstadter SP QoJulia energies are has to be equal to Hofstadter Single Particle energies.

In [4]:
using LinearAlgebra

Basis = NLevelBasis(4*4)
eigenenergies(dense(Hofstadter_SP_Op(4, 4, 1/4, Basis))) == eigvals(Hofstadter_SP(4,4,1/4,0))

true

In [6]:
function get_sub_states(SP_Op, N_cut)
    
    E0, states0 = eigenstates(dense(SP_Op))
    states = states0[1:N_cut]
    
    return states
end

get_sub_states (generic function with 1 method)

In [7]:
function get_Projector(states, Basis)
    
    b_sub = SubspaceBasis(Basis,states)
    P = projector(b_sub, Basis)
    Pt = dagger(P)
    
    return b_sub, P, Pt
end

get_Projector (generic function with 1 method)

In [8]:
function Subspace_Op(SP_Op, P)
    return P*SP_Op*Pt
end

Subspace_Op (generic function with 1 method)

In [27]:
Nx = 8; Ny = 8; N=Nx*Ny; alpha = 1/2; N_cut = 4

Basis = NLevelBasis(Nx*Ny)
H1 = Hofstadter_SP_Op(Nx, Ny, alpha, Basis)
states = get_sub_states(H1, N_cut)
b_sub, P, Pt = get_Projector(states, Basis)
H1_Projected = Subspace_Op(H1, P)

Operator(dim=4x4)
  basis: Subspace(superbasis=NLevel(N=64), states:4)
 -2.82843-0.0im       3.0e-17+0.0im      …  -9.0e-17+1.1e-16im
  5.0e-17-0.0im      -2.82843-0.0im          2.4e-16-1.3e-16im
  1.3e-16+6.0e-17im  -3.4e-16+1.0e-17im     -1.7e-16-1.7e-16im
 -8.0e-17-1.0e-17im   1.1e-16+1.9e-16im     -2.44949-1.1e-16im

In [28]:
Num_Op_List = []
Num2_Op_List = []
for m in 1:N
    NM = transition(Basis, m, m)
    NMP = P*NM*Pt
    NM2P = P*NM^2*Pt
    push!(Num2_Op_List, NM2P)
    push!(Num_Op_List, NMP)
end

In [29]:
PN = 2

MB_States = bosonstates(b_sub, [PN]) 
MB_Basis = ManyBodyBasis(b_sub, MB_States)

function get_MB_Op(MB_Basis, Basis, Op)
    
    Op_MB = SparseOperator(MB_Basis)
    for i in 1:length(Basis)
        for j in 1:length(Basis)
            Op_MB = Op_MB + Op.data[i,j] * transition(MB_Basis, i, j)
        end
    end
    
    return Op_MB
end

H1_MB = get_MB_Op(MB_Basis, b_sub, H1_Projected)

MB_Num_Op_List = []
MB_Num2_Op_List = []
for m in 1:N
    NMP = get_MB_Op(MB_Basis, b_sub, Num_Op_List[m])
    NMP2 = get_MB_Op(MB_Basis, b_sub, Num2_Op_List[m])
    push!(MB_Num_Op_List, NMP)
    push!(MB_Num2_Op_List, NMP2)
end

In [30]:
U = 2

function Int_Op(MB_Num_Op_List, MB_Num2_Op_List, U)

    IT = SparseOperator(MB_Basis)
    
    for m in 1:N
        IT = IT + U/2 * (MB_Num2_Op_List[m] - MB_Num_Op_List[m])
    end
    
    return IT
end

H_Int_MB = Int_Op(MB_Num_Op_List, MB_Num2_Op_List, U)

Operator(dim=10x10)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=64), states:4), states:10)sparse(Int64[], Int64[], ComplexF64[], 10, 10)

In [31]:
H_MB = H1_MB + H_Int_MB

Operator(dim=10x10)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=64), states:4), states:10)sparse([1, 2, 3, 4, 1, 2, 3, 4, 5, 6  …  4, 6, 7, 8, 9, 10, 4, 7, 9, 10], [1, 1, 1, 1, 2, 2, 2, 2, 2, 2  …  9, 9, 9, 9, 9, 9, 10, 10, 10, 10], ComplexF64[-5.6568542494923815 - 0.0im, 7.0e-17 - 0.0im, 1.8e-16 + 9.0e-17im, -1.1e-16 - 2.0e-17im, 4.0e-17 + 0.0im, -5.656854249492383 - 0.0im, -3.4e-16 + 1.0e-17im, 1.1e-16 + 1.9e-16im, 7.0e-17 - 0.0im, 1.3e-16 + 6.0e-17im  …  1.2e-16 - 2.0e-17im, 2.4e-16 - 1.3e-16im, -1.7e-16 + 2.0e-17im, -2.4e-16 - 2.4e-16im, -4.898979485566354 - 6.0e-17im, -2.4e-16 + 8.0e-17im, -1.2e-16 + 1.6e-16im, 3.3e-16 - 1.8e-16im, -2.4e-16 - 2.4e-16im, -4.898979485566354 - 2.2e-16im], 10, 10)

2) If U<<Band-Gap, Sub-Space Hofstadter Finite-U energies converges at Hofstadter Finite-U energies.
3) When all SP states projected to the new sub-space, Sub-Space Hofstadter Finite-U energies have to equal to Hofstadter Finite-U energies.

In [18]:
@nbinclude("Hofstadter MB in Julia.ipynb"; regex=r"#.*executeme")

Hofstadter_Non_Int (generic function with 1 method)

In [32]:
E1 = eigenenergies(dense((H_MB+dagger(H_MB))/2))

10-element Vector{Float64}:
 -5.656854249492388
 -5.6568542494923815
 -5.6568542494923815
 -5.277916867529371
 -5.27791686752937
 -5.2779168675293695
 -5.277916867529364
 -4.898979485566356
 -4.898979485566355
 -4.898979485566354

In [19]:
# using Plots

E1 = eigenenergies(dense((H_MB+dagger(H_MB))/2))

E2 = eigenenergies(dense(Hofstadter_Finite_U(Nx, Ny, alpha, PN, U)))

# print(E1-E2[1:length(E1)])

# plot(1:length(E1),E1,seriestype=:scatter,markershape=:star5, markersize=6, label="Sub-Space")
# plot!(1:length(E2),E2,seriestype=:scatter,label="Real-Space",legend=:topleft)
# xlabel!("n");ylabel!("E")

325-element Vector{Float64}:
 -5.80422606518063
 -5.804226065180629
 -5.804226065180615
 -5.804226065180608
 -5.804226065180594
 -5.6663813837786305
 -5.666381383778626
 -5.666381383778617
 -5.666381383778617
 -5.666381383778617
 -5.65929311871793
 -5.659293118717916
 -5.659293118717908
  ⋮
  5.937498269754853
  5.937498269754853
  6.183619050574752
  6.183619050574754
  6.183619050574755
  6.18361905057476
  6.183619050574766
  6.279364187452772
  6.27936418745278
  6.279364187452785
  6.279364187452792
  6.279364187452807

In [20]:
E1

15-element Vector{Float64}:
 -5.93289597828675
 -5.932895978286747
 -5.932895978286747
 -5.932895978286747
 -5.932895978286745
 -5.932895978286745
 -5.932895978286744
 -5.932895978286744
 -5.932895978286743
 -5.932895978286743
 -5.932895978286743
 -5.9328959782867425
 -5.932895978286742
 -5.932895978286742
 -5.932895978286742